In [1]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score

/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(200)

In [6]:
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2', num_labels=2).to("cuda")
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')

/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from data_module import CustomDataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
# Creating the dataset and dataloader for the neural network
df = pd.read_csv("out_data/ToxicRussianComments.csv")

MAX_LEN = max([len(comment.strip().split(' ')) for comment in df['comment']])
MAX_LEN    

267

In [8]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (248290, 2)
TRAIN Dataset: (198632, 2)
TEST Dataset: (49658, 2)


In [9]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

In [10]:
training_args = TrainingArguments(
    output_dir = './bert_results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './bert_logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=200)

In [11]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = training_set,
                  eval_dataset = testing_set,
                  compute_metrics = compute_metrics)

/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
trainer.train()

/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1
1,0.138700,0.099204,0.904767
2,0.085600,0.084746,0.921848
3,0.075600,0.085862,0.922990


/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=18624, training_loss=0.09997392929706377, metrics={'train_runtime': 2495.7919, 'train_samples_per_second': 238.76, 'train_steps_per_second': 7.462, 'total_flos': 2291538159229536.0, 'train_loss': 0.09997392929706377, 'epoch': 3.0})

In [13]:
torch.save(model.state_dict(), 'bert_ckpt.pt')

In [14]:
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('fine-tune-bert/tokenizer_config.json',
 'fine-tune-bert/special_tokens_map.json',
 'fine-tune-bert/vocab.txt',
 'fine-tune-bert/added_tokens.json')

In [15]:
def get_prediction():
    test_pred = trainer.predict(testing_set)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [21]:
print(classification_report(testing_set.targets, pred))
print(f1_score(testing_set.targets, pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     40803
           1       0.92      0.93      0.92      8855

    accuracy                           0.97     49658
   macro avg       0.95      0.95      0.95     49658
weighted avg       0.97      0.97      0.97     49658

0.9218477465818938


In [27]:
# example
example = test_dataset.loc[0:10]
print(example)


                                              comment  label
0                                скотина! что сказать      1
1             а когда мы статус агрогородка получили?      0
2   красота..!! если есть, что показать??!! почему...      0
3   эти скаты ,на героев пионеров, анекдотов навыд...      0
4                           кто заседать будет ..????      0
5   да. я на нем за 21 день марафона скинула 5кг и...      0
6                                   секс мосаж чтоли?      0
7                                        какие жалкие      0
8                самые умные быстренько развернулись!      0
9                      это же надо так научить!!!👍👍😂😂      0
10  братка поздравляю тебя с днём рождения желаю в...      0


In [28]:
example = CustomDataset(example, tokenizer, MAX_LEN)

In [30]:
example_pred = trainer.predict(example)
labels = np.argmax(example_pred.predictions, axis = -1)
print(labels)

/mnt/cs/voice/korenevskaya-a/nirma/nirma_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


[1 0 0 0 0 0 0 0 0 0 0]
